# Week 4: Support Vector Machines and Optimisation

The dataset and part of the exercises for this week were taken from MLPA.

<hr style="border:2px solid gray">

## Index: <a id='index'></a>
1. [ATLAS experiment dataset](#atlas)
2. [Linear Support Vector Machines](#linear)
3. [Support Vector Machines Optimisation](#optimisation)
4. [Feature engineering](#feature)
5. [Nested cross validation](#nestedcv)






<hr style="border:2px solid gray">

## ATLAS experiment dataset [^](#index) <a id='atlas'></a>


Let's start with our usual imports, there are a few new ones from last week.

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rc
import pandas.plotting as pdp

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder #This allows us to convert categorical features into integers
from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV # New! This will be used to explore different hyperparameter choices.
from sklearn.svm import SVC, LinearSVC # New algorithm!
from sklearn import metrics
from sklearn.pipeline import make_pipeline #This allows one to build different steps together

import UsedForMCQs

The cell below changes the output options of some of the pandas printout statements, it will make the columms more readable.

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
rc('text', usetex=False)

You know the drill now, read in the dataset using `pandas` functionality, you may want to set `index_col='ID'`. Check that the dataframe was defined correctly using `head()`. 

In [12]:
features = pd.read_csv('ParticleID_features.csv')
print(features)

        ID       MET    METphi Type_1        P1        P2        P3        P4  \
0        0   62803.5 -1.810010      j  137571.0  128444.0 -0.345744 -0.307112   
1        1   57594.2 -0.509253      j  161529.0   80458.3 -1.318010  1.402050   
2        2   82313.3  1.686840      b  167130.0  113078.0  0.937258 -2.068680   
3        3   30610.8  2.617120      j  112267.0   61383.9 -1.211050 -1.457800   
4        4   45153.1 -2.241350      j  178174.0  100164.0  1.166880 -0.018721   
...    ...       ...       ...    ...       ...       ...       ...       ...   
4995  4995  269074.0 -1.274730      j  495577.0  362590.0 -0.791914  1.671250   
4996  4996   12385.8  0.986871      j  258932.0  133559.0 -1.276540  2.970100   
4997  4997   32762.8  3.057630      b  122222.0   79947.8  0.983920 -0.399231   
4998  4998  104474.0 -1.875250      b  791028.0  457589.0  1.141530  2.934810   
4999  4999   65623.8 -0.817535      b  145360.0  138746.0  0.236765  2.277120   

     Type_2        P5      

The data frame contains 67 features and 5000 instances (you can verify this with `shape`). 

The first two columns correspond to the magnitude (MET) and azimuthal angle (METphi) of the missing transverse energy vector.

The other 65 features contain 5 descriptors (object type and energy quadrivector) for up to 13 particles observed in the event.

Just to show you how a different way of reading things work, I left the target labels in a separate file, that you can read using the line below. You may want to print `y` to verify that the reading worked correctly.

In [13]:
y = np.genfromtxt('ParticleID_labels.txt', dtype = str)

In [14]:
print(y)

['ttbar' 'ttbar' 'ttbar' ... 'ttbar' '4top' 'ttbar']


y represents an array of targets. What we want is '4top' events, our background corresponds to 'ttbar' events. This is a classification problem and you should know by now how it works.

There is a problem though, which is that at the moment this is a categorical value (string-type) and we want to convert it to numerical form (e.g. 0/1).

Use the `LabelEncoder()` `.fit_transform()` method to convert the categorical values to integers.


In [16]:
newy = LabelEncoder().fit_transform(y)

In [17]:
newy

array([1, 1, 1, ..., 1, 0, 1])

Our transformer used 1 for the first instance, but we actually wanted 4top to be the positive label, so define `target` below to flip the labels of `newy`.

In [19]:
target = newy[::-1]

In [20]:
target # Happier now.

array([1, 0, 1, ..., 1, 1, 1])

Let's take a look at these features, using the "describe" property. Note that this automatically excludes non-numerical type columns.

<div style="background-color:#C2F5DD">

## Exercise 1
1. Size: What was the size of the data set?
2. Missing data: count how many instances of missing data there are for every feature (only print out the non-zero ones).
3. Write down different options on how to deal with the missing data. (There is no need to action them yet, we will do it together later).
4. Balance: check how many `4top` events we have and compare that to the overall size of the dataset. What's the accuracy of a model that puts everything in the `ttbar` (ie negative) class? What's the accuracy of a random classifier?
5. We have a trade-off between keeping more features but having a more severe missing data/imputing problem or keeping fewer features but dealing with a simpler imputing problem. We are choosing the latter. Define a new dataframe `feature_lim` that just includes the MET variables and the first 16 columns (first four particles).
6. Count how many instances of missing data we now have for every feature, and use `.fillna` to replace the NaN values with 0s. (*Note: this is the simplest but worst possible choice - imputing a constant value skews the model :D One step up would be to input the mean or median for each column. However, because only a limited number of instances have missing data, the choice of imputing strategy doesn't matter too much*).
7. Use the `scatter_matrix` function from `pandas.plotting` (already imported at the beginning of this notebook to display a scatter matrix of the first 6 features in the dataset). Comment on the shapes of the plot.

1.

The size was substantially big 1.4MB
5000 rows x 68 columns

2.

In [29]:
target
count = 0

for i in target:
    if i == 0:
        count += 1
print(count)

811


In [24]:
print(features.isnull().sum())

ID            0
MET           0
METphi        0
Type_1        0
P1            0
P2            0
P3            0
P4            0
Type_2        3
P5            3
P6            3
P7            3
P8            3
Type_3       50
P9           50
P10          50
P11          50
P12          50
Type_4      283
P13         283
P14         283
P15         283
P16         283
Type_5      998
P17         998
P18         998
P19         998
P20         998
Type_6     2129
P21        2129
P22        2129
P23        2129
P24        2129
Type_7     3111
P25        3111
P26        3111
P27        3111
P28        3111
Type_8     3814
P29        3814
P30        3814
P31        3814
P32        3814
Type_9     4271
P33        4271
P34        4271
P35        4271
P36        4271
Type_10    4558
P37        4558
P38        4558
P39        4558
P40        4558
Type_11    4739
P41        4739
P42        4739
P43        4739
P44        4739
Type_12    4873
P45        4873
P46        4873
P47        4873
P48     

<hr style="border:2px solid gray">

## Linear Supported Vector Machines  [^](#index) <a id='linear'></a>

Linear Support Vector Machines (LinearSVC) offer a specific implementation for linear classification problems. The `C` stands for `Classifier` to distinguish them from the `Regression` models for SVMs.

In [ ]:
bmodel = LinearSVC(dual = False)

The `dual` parameter controls whether the model solves the **primal** or **dual** formulation of the linear SVM optimisation problem. 

* **Primal formulation:** Optimises the objective function directly, focusing on finding the hyperplane that maximises the margin and minimises misclassification error.
* **Dual formulation:** Introduces Lagrange multipliers and transforms the problem into a different form involving kernel functions. This can be more efficient for large datasets, especially with sparse data.

* **`dual=False`:**
    * Preferable when `n_samples > n_features` (more data points than features).
    * Offers more interpretability, as the solution directly gives the weight vector (w) defining the hyperplane.
    * Might be slower for large datasets.
* **`dual=True`:**
    * Preferable when `n_samples < n_features` (more features than data points).
    * Can be faster for large datasets, especially with sparse data.
    * Might be less interpretable, as the solution involves Lagrange multipliers instead of a direct weight vector.
* **`dual="auto"`:**
    * Convenient option that automatically chooses based on data properties.
    * Might not always be optimal, so consider manual selection based on your specific case.

In our case we prefer `dual=False` as `n_samples > n_features`. If not, will not converge!!

In [ ]:
display(UsedForMCQs.Q0)
display(UsedForMCQs.Q00)
display(UsedForMCQs.Q1)
display(UsedForMCQs.Q2)
display(UsedForMCQs.Q3)
display(UsedForMCQs.Q4)
display(UsedForMCQs.Q5)
display(UsedForMCQs.Q6)

<div style="background-color:#C2F5DD">

## Exercise 2
1. Define a cross-validation strategy called `cv`, use the shuffled `StratifiedKFold` with 5 splits and `random_state=101` for reproducibility.
2. Look at the class count to confirm it has worked as expected.
3. Calculate the scores for the train and test scores from cross validation using `cross_validate` and `accuracy`. Find their mean and standard deviation. Comment on the model performance.
4. Calculate the predicted labels and the confusion matrix, and comment on the performance of our first SVM classifier.
5. Let's try to improve the performance applying with preprocessing our set with a scaler. Define a pipeline `piped_model` which uses the `StandardSclaer()` and LinearSVC(dual = False, C = 1000)`. Then repeat the cross validation procedure and discuss the performance looking at the cross validated test/train scores and the confusion matrix.
6. Plot the learning curve for this pipeline using `cv = cv`, `train_sizes=np.array([0.05,0.1,0.2,0.5,1.0])` and `accuracy` as a score. Comment on the results.


<hr style="border:2px solid gray">

## Supported Vector Machines Optimisation [^](#index) <a id='optimisation'></a>
The above model is our **benchmark**, we ran it without optimisation (with default parameters) and now would like to optimise it.

When we optimise parameters with a grid search, we choose the parameters that give the best test scores. This is different from what would happen with new data - to do this fairly, at no point of the training procedure we are allowed to look at the test labels. Therefore, we would need to do **nested cross validation** to avoid leakage between the parameter optimisation and the cross validation procedure and properly evaluate the generalisation error.

In [ ]:
piped_model = make_pipeline(StandardScaler(), SVC()) #now using the general SVC so I can change the kernel

piped_model.get_params() #this shows how we can access parameters both for the scaler and the classifier

**We can define a dictionary of parameter values to run the optimisation.**

How do we choose the parameters values?
- Start with wide coverage (`gamma` and `C` might want to span several orders of magnitude)
- If the best parameters are found at the edge of the grid, you might want to further expand the explored range of parameters
- If the grid is too large, you can use `RandomSearchCV` instead which uses a random search.
- In principle, you could use other techniques, like MCMC, to find the best combination. However, in my experience, excessive fine tuning of the parameters is usually rewarded with poor generalisation properties (ie high variance).

Note that this might take a while (~3-5 mins); the early estimates output by this cell may be misleading because more complex models (in particular high gamma) take longer.

Once you run this cell, the "model" object will have attributes "best_score_", "best_params_" and "best_estimator_", which give us access to the optimal estimator (printed out), as well as "cv_results_" that can be used to visualise the performance of all models.

In [ ]:
#optimizing SVC: THIS IS NOT YET NESTED CV

parameters = {'svc__kernel':['poly', 'rbf'], \
              'svc__gamma':[0.00001,'scale', 0.01, 0.1], 'svc__C':[0.1, 1.0, 10.0, 100.0], \
              'svc__degree': [2, 4, 8]}

model = GridSearchCV(piped_model, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)

model.fit(features_lim,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

#### Next, we visualize the models in a data frame, and rank them according to their test scores.

I like to look at the mean and std of the test scores, the mean of the train scores (so I can evaluate if they differ and the significance of the result), and also fitting time (we may pick a faster model instead of the best model if the scores are comparable)!

In [ ]:
scores_lim = pd.DataFrame(model.cv_results_)

scores_lim.columns

In [ ]:
scores_lim[['params','mean_test_score','std_test_score','mean_train_score', \
            'mean_fit_time']].sort_values(by = 'mean_test_score', ascending = False).head(20)

To build some intuition around the results, I find it helpful to ask: **what hyperparameter values are common to all the best-performing models?**

Here, for example, the rbf kernel seems to be constantly preferred, while the values of C and gamma seem to only affect the scores only mildly. Note also that the Grid Search is insensitive to moot parameters combinations; for example, here the first three models are identical, because the degree of the polynomial kernel does not matter when using an rbf kernel. This is less than ideal, of course.

#### We can also isolate one type of kernel to look at it more closely.

In [ ]:
scores_lim[scores_lim['param_svc__kernel'] == 'poly'][['params','mean_test_score','std_test_score',\
                        'mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', ascending = False).head(10)

### Final diagnosis 

The problem here is high bias, which is not that surprising given that we are using only a subset of features.

We can try two things: 
- using an imputing strategy to include information about the discarded features.
- making up new features which might help, based on what we know about the problem.
  

In [ ]:
display(UsedForMCQs.Q7)
display(UsedForMCQs.Q8)
display(UsedForMCQs.Q9)
display(UsedForMCQs.Q10)

<hr style="border:2px solid gray">

## Feature engineering  [^](#index) <a id='feature'></a>

<div style="background-color:#C2F5DD">

## Exercise 3
1. Let's go back to our original dataframe `features` and try using all of the features. Fill all NaN values with 0, and all empty categorical values with 0 too. (*Hint: for the second task you may want to use `.replace`*).
2. Look at what kind of particles are produced in a collision. *Hint: you can do that by using `np.unique` on the values of the particle type features*. You should find 7 unique particles + the 0 for when there is no particle.
3. **Feature engineering 1**: Calculate the total number of products of each type for example for b-jets:
`nbtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'b' for i in range(1,14)])) for j in range(features.shape[0])])`. Add the following new features to the dataframe:
    1. The total number of particles produced `Total_products`
    2. The total number of b jets `Total_b`
    3. The total number of jets `Total_j`
    4. The total number of leptons (electrons, positron, mu+, mu-) `Total_leptons`

4. New define a new dataframe `features_lim_2` which contains the two MET variables, the 16 features of the first 4 products, and the total features we just calculated above.
5. Define a pipeline `piped_model` which uses the `StandardScaler()` and `LinearSVC(dual = False)`. Then repeat the cross validation procedure and discuss the performance looking at the cross validated test/train scores, the confusion matrix, and the learning curve for `accuracy` like we did above. Comment on the results.
6. Define a new pipeline `piped_model` using `StandardScaler()` and the general `SVC()` class so that you can optimise the model using a grid search of hyperparametes like we did above. Which is the best model and why?

### Another feature engineering attempt we could potentially do is use the type of product in the i-th location as a feature.

We could do it with label encoding, as we did earlier in this notebook, but such strategy introduces a notion of distance metric (labels that are mapped to 0 and 1 are interpreted to be closer to each other than labels that are mapped into 0 and 7). 

As an alternative, we can introduce as many new columns as possible values for each categorical variable we are re-mapping, and we just use a 0/1 to indicate that the particle is of that type. This is achieved with the wonderfully-named "get_dummies" function:

In [ ]:
features_add = pd.get_dummies(data=features, columns=['Type_'+str(i) for i in range(1,14)])

In [ ]:
features_add.columns[58:80] #A subset of the new features

In [ ]:
features_add.shape

**Feature engineering 2:** add other variables (type of product) for the first four particles.

In [ ]:
features_lim_3 = features_add[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16','Total_products', 'Total_b' ,'Total_j','Total_g', 
              'Total_leptons','Type_1_b',
       'Type_1_j', 'Type_2_0', 'Type_2_b', 'Type_2_e+', 'Type_2_e-',
       'Type_2_g', 'Type_2_j', 'Type_2_m+', 'Type_2_m-', 'Type_3_0',
       'Type_3_b', 'Type_3_e+', 'Type_3_e-', 'Type_3_g', 'Type_3_j',
       'Type_3_m+', 'Type_3_m-', 'Type_4_0', 'Type_4_b', 'Type_4_e+',
       'Type_4_e-', 'Type_4_g', 'Type_4_j', 'Type_4_m+', 'Type_4_m-']]

In [ ]:
features_lim_3.head()

<div style="background-color:#C2F5DD">

## Exercise 4
1. Define a pipeline `piped_model` which uses the `StandardSclaer()` and `LinearSVC(dual = False)`. Then repeat the cross validation procedure on the new data frame `features_lim_3` and discuss the performance looking at the cross validated test/train scores, the confusion matrix, and the learning curve for `accuracy` like we did above. Comment on the results.
2. Define a new pipeline `piped_model` using `StandardScaler()` and the general `SVC()` class so that you can optimise the model using a grid search of hyperparameters like we did above. Comment on the results.
3. Define yet a new pipeline and run the cross validation procedure using all features (ie `features_add`). Discuss the performance looking at cross validates test/train scores, confusion matrix, and learning curve for `accuracy`. Comment on the results.

<hr style="border:2px solid gray">

## Nested cross validation   <a id='nestedcv'></a> [^](#index)

It is clear from our exploration that the model that performed best is the one using `features_lim_2`. Our `GridSearchCV` also gave us an idea of which parameters would be useful to further explore.

Now we want to get an estimate of the generalisation error using **nested cross validation**. This can be computationally very expensive, so it's worth it to first select carefully the parameter grid we are going to explore. 


In [ ]:
display(UsedForMCQs.Q11)
display(UsedForMCQs.Q12)
display(UsedForMCQs.Q13)
display(UsedForMCQs.Q14)

<div style="background-color:#C2F5DD">

## Exercise 5
1. Define the hyperparameters for the nested cross validation based on the results of the hyperparameter searches above. Suggestion: use a number of parameters that results in less than 20 total combinations
2. Define a `GridSearchCV` using these parameters and shuffled `StratifiedkFold` with 4 splits.
3. Calculate the scores of nested cross validation using the `cross_val_score` function and a shuffled `StratifiedkFold` with 4 splits.
4. Calculate the mean and standard deviation of the scores. Comment on the results.